#### ver6 : 유사 값 분류를 통한 K-means 진행 

- columns : 외부 온도, 차량 속도, 일사량, 습도, 내부온도(8개 중 1개씩)
- xlsx : cross_tab_columns_5, clustering_5columns_k_3

In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
csv_path = "/home/miserver10/workspace_vscode/datasets/data/*.csv"

csv_files = glob.glob(csv_path)

df_list = [pd.read_csv(csv_file) for csv_file in csv_files]
df = pd.concat(df_list, ignore_index=True)  #ignore_index = 기존 인덱스 무시하여 새로운 인덱스 생성(연속으로 이어진다)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 54 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Front_Seat_LH_Temp Unit: Degree ('c)     339932 non-null  float64
 1   Front_Seat_RH_Temp Unit: Degree('c)      339932 non-null  float64
 2   Rear_Seat_CT_Temp Unit: Degree('c)       339932 non-null  float64
 3   Front_Roof_CT_Temp Unit: Degree('c)      339932 non-null  float64
 4   Front_Roof_LH_Temp Unit: Degree('c)      339932 non-null  float64
 5   Front_Roof_RH_Temp Unit: Degree('c)      339932 non-null  float64
 6   Rear_Roof_LH_Temp Unit: Degree('c)       339932 non-null  float64
 7   Rear_Roof_RH_Temp Unit: Degree('c)       339932 non-null  float64
 8   DRV_LH_Vent_Wind_Velocity Unit: m / s    339932 non-null  float64
 9   DRV_RH_Vent_Wind_Velocity Unit: m / s    339932 non-null  float64
 10  DRV_Foot_Vent_Wind_Velocity Unit

In [3]:
drop_list = ["DRV_LH_Vent_Wind_Velocity Unit: m / s",
       "DRV_RH_Vent_Wind_Velocity Unit: m / s",
       "DRV_Foot_Vent_Wind_Velocity Unit: m / s",
       "DRV_Seat_Vent_Wind_Velocity Unit: m / s",
       "PAS_LH_Vent_Wind_Velocity Unit: m / s",
       "PAS_RH_Vent_Wind_Velocity Unit: m / s",
       "PAS_Foot_Vent_Wind_Velocity Unit: m / s",
       "PAS_Seat_Vent_Wind_Velocity Unit: m / s", "5V_ADC_CH1 Unit: V",
       "5V_ADC_CH2 Unit: V", "5V_ADC_CH3 Unit: V", "5V_ADC_CH4 Unit: V",
       "5V_ADC_CH5 Unit: V", "PWM_CH8 Unit: PWM", "Frequency_CH8 Unit: Hz",
       "CR_Datc_DrModeAct","CR_Datc_TempAct", "CR_Datc_IntakeAct", "CR_Datc_AdsAct",
       "CR_Datc_DrVentTemp_C", "CR_Datc_DrFloorTemp_C", 
       "CR_Datc_ClimatePowSum_W", "LAT_ACCEL_m/s^2", "LONG_ACCEL_m/s^2",
       "YAW_RATE_'/s", "WHL_SPD_FL_km/h", "WHL_SPD_FR_km/h", "WHL_SPD_RL_km/h",
       "WHL_SPD_RR_km/h", 'CR_Datc_AcnCompSpdFB_rpm', 'CR_Datc_AptVal_PSI', 'CR_Datc_OutTemp_C', 
       'CR_Datc_OutTempC' ,'CR_Datc_EvapSnsTemp_C', 'CR_Datc_DrTempDispC', 'DATC_ModeDisp']

df = df.drop(drop_list, axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 18 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Front_Seat_LH_Temp Unit: Degree ('c)  339932 non-null  float64
 1   Front_Seat_RH_Temp Unit: Degree('c)   339932 non-null  float64
 2   Rear_Seat_CT_Temp Unit: Degree('c)    339932 non-null  float64
 3   Front_Roof_CT_Temp Unit: Degree('c)   339932 non-null  float64
 4   Front_Roof_LH_Temp Unit: Degree('c)   339932 non-null  float64
 5   Front_Roof_RH_Temp Unit: Degree('c)   339932 non-null  float64
 6   Rear_Roof_LH_Temp Unit: Degree('c)    339932 non-null  float64
 7   Rear_Roof_RH_Temp Unit: Degree('c)    339932 non-null  float64
 8   CR_Datc_AcnCompPwrCon_W               339932 non-null  int64  
 9   CR_Datc_OutTempSnr_C                  339932 non-null  float64
 10  CR_Datc_PTCPwrCon_W                   339932 non-null  int64  
 11  

In [5]:
df.rename(columns={'CR_Datc_OutTempSnr_C': '외부 온도', 'CR_Datc_AcnCompPwrCon_W': '에어컨 압축기 전력 소비량',
                   'CR_Datc_PTCPwrCon_W': '히터의 전력 소비량', 'DATC_BlowerLevel': '바람세기', 'CF_Datc_Heatpump_Mode' : '히트 펌프',
                   'CR_Datc_SetTemp_C': '설정 온도', 'CR_Datc_IncarSnsTemp_C': '내부 온도', 
                   'CR_Datc_PhotoSnsFB_V' : '일사량', 'CR_Datc_Humidity' : '습도', 'CR_Mcu_VehSpd_Kph_sum' : '차량 속도',
                   "Front_Seat_LH_Temp Unit: Degree ('c)" : '운전석 온도', "Front_Seat_RH_Temp Unit: Degree('c)" : '보조석 온도', 
                   "Rear_Seat_CT_Temp Unit: Degree('c)" : '뒷자석 온도(중앙)', "Front_Roof_CT_Temp Unit: Degree('c)" : '앞 루프 중앙',
                   "Front_Roof_LH_Temp Unit: Degree('c)": '앞 루프 좌측', "Front_Roof_RH_Temp Unit: Degree('c)" : '앞 루프 우측',
                   "Rear_Roof_LH_Temp Unit: Degree('c)": '뒷 루프 좌측', "Rear_Roof_RH_Temp Unit: Degree('c)": '뒷 루프 우측'}, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   운전석 온도          339932 non-null  float64
 1   보조석 온도          339932 non-null  float64
 2   뒷자석 온도(중앙)      339932 non-null  float64
 3   앞 루프 중앙         339932 non-null  float64
 4   앞 루프 좌측         339932 non-null  float64
 5   앞 루프 우측         339932 non-null  float64
 6   뒷 루프 좌측         339932 non-null  float64
 7   뒷 루프 우측         339932 non-null  float64
 8   에어컨 압축기 전력 소비량  339932 non-null  int64  
 9   외부 온도           339932 non-null  float64
 10  히터의 전력 소비량      339932 non-null  int64  
 11  히트 펌프           339932 non-null  int64  
 12  설정 온도           339932 non-null  float64
 13  내부 온도           339932 non-null  float64
 14  바람세기            339932 non-null  int64  
 15  일사량             339932 non-null  float64
 16  습도              339932 non-null  float64
 17  차량 속도     

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
운전석 온도,339932.0,22.075854,3.695042,1.9,20.000000,22.400000,24.500000,32.100000
보조석 온도,339932.0,21.753655,3.734846,3.3,19.700000,22.000000,24.200000,32.300000
뒷자석 온도(중앙),339932.0,20.945707,3.323121,4.0,19.000000,21.100000,23.100000,30.800000
앞 루프 중앙,339932.0,24.860887,4.728733,6.7,21.900000,24.100000,26.700000,46.500000
앞 루프 좌측,339932.0,25.304704,4.832472,7.9,22.300000,24.600000,27.300000,46.700000
앞 루프 우측,339932.0,24.664105,5.166961,7.5,21.600000,23.800000,26.400000,48.000000
뒷 루프 좌측,339932.0,24.299146,4.914812,6.5,21.200000,23.400000,26.400000,44.900000
뒷 루프 우측,339932.0,24.242117,4.495277,7.0,21.600000,23.700000,26.200000,44.000000
에어컨 압축기 전력 소비량,339932.0,253.707359,299.143430,0.0,0.000000,200.000000,420.000000,2010.000000
외부 온도,339932.0,18.767186,10.624092,-9.0,11.000000,20.500000,26.500000,39.000000


In [8]:
df.columns

Index(['운전석 온도', '보조석 온도', '뒷자석 온도(중앙)', '앞 루프 중앙', '앞 루프 좌측', '앞 루프 우측',
       '뒷 루프 좌측', '뒷 루프 우측', '에어컨 압축기 전력 소비량', '외부 온도', '히터의 전력 소비량', '히트 펌프',
       '설정 온도', '내부 온도', '바람세기', '일사량', '습도', '차량 속도'],
      dtype='object')

In [9]:
df_temp = df[['운전석 온도', '보조석 온도', '뒷자석 온도(중앙)', '앞 루프 중앙', '앞 루프 좌측', '앞 루프 우측','뒷 루프 좌측', '뒷 루프 우측']]

df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   운전석 온도      339932 non-null  float64
 1   보조석 온도      339932 non-null  float64
 2   뒷자석 온도(중앙)  339932 non-null  float64
 3   앞 루프 중앙     339932 non-null  float64
 4   앞 루프 좌측     339932 non-null  float64
 5   앞 루프 우측     339932 non-null  float64
 6   뒷 루프 좌측     339932 non-null  float64
 7   뒷 루프 우측     339932 non-null  float64
dtypes: float64(8)
memory usage: 20.7 MB


In [10]:
df = df.drop(['운전석 온도', '보조석 온도', '뒷자석 온도(중앙)', '앞 루프 중앙', '앞 루프 좌측', '앞 루프 우측','뒷 루프 좌측', '뒷 루프 우측'], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   에어컨 압축기 전력 소비량  339932 non-null  int64  
 1   외부 온도           339932 non-null  float64
 2   히터의 전력 소비량      339932 non-null  int64  
 3   히트 펌프           339932 non-null  int64  
 4   설정 온도           339932 non-null  float64
 5   내부 온도           339932 non-null  float64
 6   바람세기            339932 non-null  int64  
 7   일사량             339932 non-null  float64
 8   습도              339932 non-null  float64
 9   차량 속도           339932 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 25.9 MB


In [11]:
# 외부 온도: 4도씩 범주화
bins_temp = np.arange(-9, 41, 4)
labels_temp = [f'{i}-{i+4}' for i in bins_temp[:-1]]
df['외부온도범위'] = pd.cut(df['외부 온도'], bins=bins_temp, labels=labels_temp, right=False, include_lowest=True)

# 차량 속도: 50km/h씩 범주화
bins_speed = [0, 50, 100, float('inf')]  # 0-50, 50-100, 100 이상
labels_speed = ['0-50', '50-100', '100+']  # 레이블 설정
df['차량속도범위'] = pd.cut(df['차량 속도'], bins=bins_speed, labels=labels_speed, right=False, include_lowest=True)

# 일사량: 0.4씩 범주화
bins_solar = np.arange(0, 1.24 + 0.4, 0.4)
labels_solar = [f'{i:.1f}-{i+0.4:.1f}' for i in bins_solar[:-1]]
df['일사량범위'] = pd.cut(df['일사량'], bins=bins_solar, labels=labels_solar, right=False, include_lowest=True)

# 습도: 20씩 범주화
bins_humidity = np.arange(0, 101, 20)
labels_humidity = [f'{i}-{i+20}' for i in bins_humidity[:-1]]
df['습도범위'] = pd.cut(df['습도'], bins=bins_humidity, labels=labels_humidity, right=False, include_lowest=True)


In [12]:
# 결과 출력
print(df.head())

   에어컨 압축기 전력 소비량  외부 온도  히터의 전력 소비량  히트 펌프  설정 온도  내부 온도  바람세기   일사량    습도  \
0             420   22.5           0      0   18.0   27.5     4  0.48  30.0   
1             420   22.5           0      0   18.0   27.5     4  0.48  30.0   
2             420   22.5           0      0   18.0   27.5     4  0.48  30.0   
3             420   22.5           0      0   18.0   27.5     4  0.46  30.0   
4             420   22.5           0      0   18.0   27.5     4  0.46  30.0   

   차량 속도 외부온도범위 차량속도범위    일사량범위   습도범위  
0    0.0  19-23   0-50  0.4-0.8  20-40  
1    0.0  19-23   0-50  0.4-0.8  20-40  
2    0.0  19-23   0-50  0.4-0.8  20-40  
3    0.0  19-23   0-50  0.4-0.8  20-40  
4    0.0  19-23   0-50  0.4-0.8  20-40  


In [13]:
# 선택된 열만 범주화할 수 있는 함수
def categorize_columns(df, interval=4, selected_columns=None):
    categorized_df = df.copy()
    
    # 선택된 열이 없으면 모든 열을 선택
    if selected_columns is None:
        selected_columns = df.columns
    
    for col in selected_columns:
        if col in df.columns:  # 열이 데이터프레임에 존재하는지 확인
            min_val = df[col].min()
            max_val = df[col].max()
            bins = np.round(np.arange(min_val, max_val + interval, interval), 1)
            labels = [f'{i}-{i+interval}' for i in bins[:-1]]
            categorized_df[col + '_범위'] = pd.cut(df[col], bins=bins, labels=labels, right=False, include_lowest=True)
    
    return categorized_df

# 특정 열만 범주화
selected_columns = ['운전석 온도', '보조석 온도', '뒷자석 온도(중앙)', '앞 루프 중앙', '앞 루프 좌측', '앞 루프 우측', '뒷 루프 좌측', '뒷 루프 우측']
df_categorized = categorize_columns(df_temp, interval=4, selected_columns=selected_columns)

In [14]:
df_categorized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   운전석 온도         339932 non-null  float64 
 1   보조석 온도         339932 non-null  float64 
 2   뒷자석 온도(중앙)     339932 non-null  float64 
 3   앞 루프 중앙        339932 non-null  float64 
 4   앞 루프 좌측        339932 non-null  float64 
 5   앞 루프 우측        339932 non-null  float64 
 6   뒷 루프 좌측        339932 non-null  float64 
 7   뒷 루프 우측        339932 non-null  float64 
 8   운전석 온도_범위      339932 non-null  category
 9   보조석 온도_범위      339932 non-null  category
 10  뒷자석 온도(중앙)_범위  339932 non-null  category
 11  앞 루프 중앙_범위     339932 non-null  category
 12  앞 루프 좌측_범위     339932 non-null  category
 13  앞 루프 우측_범위     339932 non-null  category
 14  뒷 루프 좌측_범위     339932 non-null  category
 15  뒷 루프 우측_범위     339932 non-null  category
dtypes: category(8), float64(8)
memory usage: 23.3 MB


In [15]:
df_categorized.columns

Index(['운전석 온도', '보조석 온도', '뒷자석 온도(중앙)', '앞 루프 중앙', '앞 루프 좌측', '앞 루프 우측',
       '뒷 루프 좌측', '뒷 루프 우측', '운전석 온도_범위', '보조석 온도_범위', '뒷자석 온도(중앙)_범위',
       '앞 루프 중앙_범위', '앞 루프 좌측_범위', '앞 루프 우측_범위', '뒷 루프 좌측_범위', '뒷 루프 우측_범위'],
      dtype='object')

In [16]:
combined_df = pd.concat([df, df_categorized[['보조석 온도', '보조석 온도_범위']]], axis = 1)

# 이 중에서 골라서 사용하기 
# '운전석 온도', '운전석 온도_범위'
# '보조석 온도', '보조석 온도_범위'
# '뒷자석 온도(중앙)', '뒷자석 온도(중앙)_범위'
# '앞 루프 중앙', '앞 루프 중앙_범위'
# '앞 루프 좌측', '앞 루프 좌측_범위'
# '앞 루프 우측', '앞 루프 우측_범위'
# '뒷 루프 좌측', '뒷 루프 좌측_범위'
# '뒷 루프 우측', '뒷 루프 우측_범위'

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   에어컨 압축기 전력 소비량  339932 non-null  int64   
 1   외부 온도           339932 non-null  float64 
 2   히터의 전력 소비량      339932 non-null  int64   
 3   히트 펌프           339932 non-null  int64   
 4   설정 온도           339932 non-null  float64 
 5   내부 온도           339932 non-null  float64 
 6   바람세기            339932 non-null  int64   
 7   일사량             339932 non-null  float64 
 8   습도              339932 non-null  float64 
 9   차량 속도           339932 non-null  float64 
 10  외부온도범위          339931 non-null  category
 11  차량속도범위          339932 non-null  category
 12  일사량범위           339932 non-null  category
 13  습도범위            339932 non-null  category
 14  보조석 온도          339932 non-null  float64 
 15  보조석 온도_범위       339932 non-null  category
dtypes: category(5), float64(7), int64(4)
m

In [17]:
combined_df.columns

Index(['에어컨 압축기 전력 소비량', '외부 온도', '히터의 전력 소비량', '히트 펌프', '설정 온도', '내부 온도',
       '바람세기', '일사량', '습도', '차량 속도', '외부온도범위', '차량속도범위', '일사량범위', '습도범위',
       '보조석 온도', '보조석 온도_범위'],
      dtype='object')

In [18]:
new_order = ['에어컨 압축기 전력 소비량', '히터의 전력 소비량', '히트 펌프', '외부 온도', '보조석 온도', '설정 온도', '바람세기', '일사량', 
             '습도', '차량 속도', '내부 온도', '외부온도범위', '차량속도범위', '일사량범위', '습도범위', '보조석 온도_범위']

combined_df = combined_df[new_order]

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   에어컨 압축기 전력 소비량  339932 non-null  int64   
 1   히터의 전력 소비량      339932 non-null  int64   
 2   히트 펌프           339932 non-null  int64   
 3   외부 온도           339932 non-null  float64 
 4   보조석 온도          339932 non-null  float64 
 5   설정 온도           339932 non-null  float64 
 6   바람세기            339932 non-null  int64   
 7   일사량             339932 non-null  float64 
 8   습도              339932 non-null  float64 
 9   차량 속도           339932 non-null  float64 
 10  내부 온도           339932 non-null  float64 
 11  외부온도범위          339931 non-null  category
 12  차량속도범위          339932 non-null  category
 13  일사량범위           339932 non-null  category
 14  습도범위            339932 non-null  category
 15  보조석 온도_범위       339932 non-null  category
dtypes: category(5), float64(7), int64(4)
m

In [19]:
cross_tab = pd.crosstab(index=[combined_df['외부온도범위'], combined_df['보조석 온도_범위'], combined_df['차량속도범위'], combined_df['일사량범위']], columns=combined_df['습도범위'])

cross_tab

습도범위                             0-20  20-40  40-60  60-80  80-100
외부온도범위 보조석 온도_범위 차량속도범위 일사량범위                                     
-9--5  3.3-7.3   0-50   0.0-0.4     0     10    185      0       0
       7.3-11.3  0-50   0.0-0.4     0    247      3      0       0
       11.3-15.3 0-50   0.0-0.4     0    874      0      0       0
                        0.4-0.8     0    132      0      0       0
       15.3-19.3 0-50   0.0-0.4     0    132     96     22       0
...                               ...    ...    ...    ...     ...
35-39  23.3-27.3 100+   0.0-0.4     0     21      0      0       0
                        0.4-0.8   113     24      0      0       0
       27.3-31.3 0-50   0.0-0.4     0     29      0      0       0
                        0.4-0.8   196      0      0      0       0
                        0.8-1.2   125      0      0      0       0

[341 rows x 5 columns]

In [20]:
pivot_table = pd.pivot_table(
    combined_df,  
    index=['외부온도범위', '보조석 온도_범위', '차량속도범위', '일사량범위' ],
    columns='습도범위',
    aggfunc='size',  # count를 원하면 'size', 특정 통계치를 원하면 'np.mean' 등 사용 가능
    fill_value=0
)

pivot_table

습도범위                             0-20  20-40  40-60  60-80  80-100
외부온도범위 보조석 온도_범위 차량속도범위 일사량범위                                     
-9--5  3.3-7.3   0-50   0.0-0.4     0     10    185      0       0
                        0.4-0.8     0      0      0      0       0
                        0.8-1.2     0      0      0      0       0
                        1.2-1.6     0      0      0      0       0
                 50-100 0.0-0.4     0      0      0      0       0
...                               ...    ...    ...    ...     ...
35-39  31.3-35.3 50-100 1.2-1.6     0      0      0      0       0
                 100+   0.0-0.4     0      0      0      0       0
                        0.4-0.8     0      0      0      0       0
                        0.8-1.2     0      0      0      0       0
                        1.2-1.6     0      0      0      0       0

[1152 rows x 5 columns]

In [21]:
excel_filename = 'cross_tab_columns_5.xlsx'
cross_tab.to_excel(excel_filename)

print(f"교차 테이블 결과가 '{excel_filename}' 파일로 저장되었습니다.")

교차 테이블 결과가 'cross_tab_columns_5.xlsx' 파일로 저장되었습니다.


In [22]:
# 카테고리별 데이터 저장을 위한 딕셔너리 생성
category_data = {}

# 모든 카테고리 조합에 대해 데이터프레임을 생성
for temp_range in combined_df['외부온도범위'].unique():
    for dr_temp_range in combined_df['보조석 온도_범위'].unique():
        for speed_range in combined_df['차량속도범위'].unique():
            for solar_range in combined_df['일사량범위'].unique():
                for humidity_range in combined_df['습도범위'].unique():
                    # 각 카테고리 조합에 해당하는 데이터프레임 생성
                    subset = combined_df[
                    (combined_df['외부온도범위'] == temp_range) &
                    (combined_df['보조석 온도_범위'] == dr_temp_range) &
                    (combined_df['차량속도범위'] == speed_range) &
                    (combined_df['일사량범위'] == solar_range) &
                    (combined_df['습도범위'] == humidity_range)]
                    subset = subset.drop(['외부온도범위', '보조석 온도_범위', '차량속도범위','일사량범위','습도범위'], axis=1)  # 카테고리 이름 열 제외
                    if len(subset) >= 30 :  # 데이터가 있는지 확인
                        category_data[f'tp_{temp_range}_sp_{speed_range}_'] = subset

# # 딕셔너리에 저장된 데이터프레임 출력
# for key, value in category_data.items():
#     print(f"카테고리: {key}")
#     print(value)
#     print()  

In [23]:
print(f"category_data의 크기: {len(category_data)}")

category_data의 크기: 35


In [24]:
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

# # 스케일러 정의
# scaler = StandardScaler()

# # 각 카테고리별로 KMeans를 적용하고 결과를 저장할 딕셔너리 생성
# clustered_data = {}

# for key, value in category_data.items():
#     # 선택한 특성에 대해 스케일링 수행
#     scaled_value = value.copy()  # 원본 데이터 복사
#     scaled_value = scaler.fit_transform(scaled_value)
    
#     # KMeans 적용
#     kmeans = KMeans(n_clusters=3, random_state=42)
#     clusters = kmeans.fit_predict(scaled_value)
    
#     # 클러스터별 평균 계산 (스케일링 되기 전의 값으로)
#     clustered_value = value.copy()
#     clustered_value['클러스터'] = clusters
#     cluster_means = clustered_value.groupby('클러스터').mean()
    
#     # 결과 저장
#     clustered_data[key] = cluster_means

# # 클러스터링 결과를 CSV 파일로 저장
# for key, value in clustered_data.items():
#     value.to_csv(f'clustered_results_{key}.csv')
#     print(f"카테고리 '{key}'에 대한 클러스터링 결과가 'clustered_results_{key}.csv' 파일로 저장되었습니다.")

In [25]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 스케일러 정의
scaler = StandardScaler()

# 결과를 저장할 엑셀 파일 이름
excel_file = 'clustering_5columns_k_3.xlsx'

# 엑셀 파일 생성
with pd.ExcelWriter(excel_file) as writer:
    for key, value in category_data.items():
        # 외부 온도, 차량 속도, 일사량, 습도 열만 선택하여 클러스터링 수행
        features = ['외부 온도', '차량 속도', '일사량', '습도', '보조석 온도']
        selected_features = value[features]
        scaled_value = scaler.fit_transform(selected_features)

        # KMeans 적용
        kmeans = KMeans(n_clusters=3, random_state=42)
        clusters = kmeans.fit_predict(scaled_value)

        # 클러스터링 결과를 데이터프레임에 추가
        clustered_value = value.copy()
        clustered_value['클러스터'] = clusters

        # 클러스터별 평균 계산 (스케일링 되기 전의 값으로)
        cluster_means = clustered_value.groupby('클러스터').mean()

        cluster_means = cluster_means.round(1)

        # 결과를 엑셀 파일의 시트로 저장
        cluster_means.to_excel(writer, sheet_name=key)

# 엑셀 파일 저장 완료 메시지 출력
print(f"클러스터링 결과가 '{excel_file}' 파일에 저장되었습니다.")

/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.

클러스터링 결과가 'clustering_5columns_k_3.xlsx' 파일에 저장되었습니다.


/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
